In [2]:
import numpy as np
from enum import StrEnum
from itertools import pairwise

In [3]:
# loading map data into numpy array
with open('map.txt') as f:
    map = f.readlines()

map = [x.strip() for x in map]
map = [list(x) for x in map]
map = np.array(map)

In [4]:
# directions
class MapMarkers(StrEnum):
    UP = 'U'
    RIGHT = 'R'
    LEFT = 'L'
    DOWN = 'D'
    VISITED = 'X'
    START = '^'
    OBSTACLE = '#'

In [13]:
def get_next_coordinates(
    map: np.array, current_coordinates: tuple[int], direction: str
) -> tuple[tuple[int, int], str, bool]:
    direction_map = {
        MapMarkers.UP: (-1, 0),
        MapMarkers.DOWN: (1, 0),
        MapMarkers.LEFT: (0, -1),
        MapMarkers.RIGHT: (0, 1),
    }

    next_direction_map = {
        MapMarkers.UP: MapMarkers.RIGHT,
        MapMarkers.RIGHT: MapMarkers.DOWN,
        MapMarkers.DOWN: MapMarkers.LEFT,
        MapMarkers.LEFT: MapMarkers.UP,
    }

    while True:
        delta = direction_map[direction]
        next_coordinates = (
            current_coordinates[0] + delta[0],
            current_coordinates[1] + delta[1],
        )

        # check for out of bounds:
        if (
            next_coordinates[0] < 0
            or next_coordinates[1] < 0
            or next_coordinates[0] >= map.shape[0]
            or next_coordinates[1] >= map.shape[1]
        ):
            return next_coordinates, direction, True

        if map[next_coordinates] == MapMarkers.OBSTACLE:
            direction = next_direction_map[direction]
            continue
        else:
            return next_coordinates, direction, False

In [14]:
np.where(map == MapMarkers.START)

(array([69]), array([74]))

In [29]:
# find starting coordinates
current_coordindates = (69, 74)
direction = MapMarkers.UP
steps = [(current_coordindates, direction)]

while True:
    next_coordinates, direction, oob = get_next_coordinates(map, current_coordindates, direction)
    steps.append((next_coordinates, direction))
    # checking if coordinates are outside of the map
    if oob:
        break
    else:
        current_coordindates = next_coordinates

In [30]:
len(steps)

6006

In [31]:
map_steps = map.copy()

for step, _ in steps:
    map_steps[step] = MapMarkers.VISITED

for row in map_steps:
    print(''.join(row))

....#.................#......................#..........................#..................#....##..#....X......#.................
...................................#...............................#......#..#...........................X...#....................
..........................#................#......##.....#.....................................#.........X.....#..#...............
..........................XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX#.X........................
...................#....#.X.......................#..............#.....#..............................XXXXXXXXXXXXXXXXXXXXXXXX#...
.....#.............XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX#.............X....
...................X......X..............................#....................#..#............#.......XX.X....X#.............X....
..............#....X......X...#..........................XXXXXXXXXXXXXXXXXXXXXXXXXX

In [33]:
np.unique(map_steps, return_counts=True)

(array(['#', '.', 'X'], dtype='<U1'), array([  785, 10653,  5462]))

In [35]:
# subtracting 1 because we don't want to count the starting position
len({c for c, _ in steps}) - 1

5461

In [38]:
# Part 2
# need to check where in the steps I need to put an obstacle to create an infinite loop
# visting the same coordinates with the same direction should imply an infinte loop

infinite_loops = 0
path = steps[:-1]
path_visited = set()

for (coordinates, direction), (next_step, _) in pairwise(path):
    path_visited.add(coordinates)
    current_path = set() | path_visited

    # temporarliy add obstacle
    original_value = map[next_step]
    map[next_step] = MapMarkers.OBSTACLE

    while True:
        next_coordinates, next_direction, oob = get_next_coordinates(
            map, coordinates, direction
        )

        # if we have visited the next coordinates with the same direction, we have an infinite loop
        if (next_coordinates, next_direction) in current_path:
            infinite_loops += 1
            break

        # checking if coordinates are outside of the map
        if oob:
            break

        current_path.add((next_coordinates, next_direction))
        coordinates = next_coordinates
        direction = next_direction

    # remove obstacle
    map[next_step] = original_value

print(infinite_loops)

2020
